In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier

mnist = fetch_openml('mnist_784')

X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.2, random_state=42)

knn = KNeighborsClassifier()

param_grid = {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']}
grid_search = GridSearchCV(knn, param_grid, cv=5, verbose=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Accuracy: ", grid_search.score(X_test, y_test))
print("Best hyperparameters: ", grid_search.best_params_)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import shift

def shift_image(image: np.ndarray, dx: int, dy: int) -> np.ndarray:
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0)
    return shifted_image.flatten()
  
def plot_image(image: np.ndarray, title: str, image_order: int) -> None:
    plt.subplot(130 + image_order)
    plt.title(title)
    plt.imshow(image.reshape(28, 28), interpolation="nearest", cmap="Greys")


plt.figure(figsize=(12, 3))

image = X_train.values[125]
plot_image(image, "Original", 1)

shifted_image_left = shift_image(image, -5, 0)
plot_image(shifted_image_left, "Shifted left", 2)

shifted_image_up = shift_image(image, 0, -5)
plot_image(shifted_image_up, "Shifted up", 3)


plt.show()

In [ ]:
X_train_augmented = [image for image in X_train.values]
y_train_augmented = [label for label in y_train.values]

for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    for image, label in zip(X_train.values, y_train.values):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

In [ ]:
knn_clf = KNeighborsClassifier(**grid_search.best_params_)
knn_clf.fit(X_train_augmented, y_train_augmented)

print("Accuracy: ", knn_clf.score(X_test, y_test))